In [125]:
import numpy as np
from scipy.io import loadmat
from scipy import optimize

reg = 1
labels = 10
data = loadmat('ex3data1.mat')
X = data['X']
y = data['y']
y[y == 10] = 0
# X : 5000x400
# y : 5000x1
# Initialise theta matrices
weights = loadmat('ex4weights.mat')
theta1 = weights['Theta1']
# theta1 : 25x401
theta2 = weights['Theta2']
# theta2 : 10x26

In [127]:
def sigmoid(mat):
    return (1/(1 + np.exp(-mat)))

In [126]:
def sigmoidGrad(mat):
    return sigmoid(mat)*(1 - sigmoid(mat))

In [128]:
def forwardPropagation(theta, X):
    theta1 = np.reshape(theta[0:10025], (25, 401))
    theta2 = np.reshape(theta[10025:10285], (10, 26))
    
    X_bias = np.concatenate((np.ones((5000,1)), X), axis=1)
    # 25x5000
    g2 = np.matmul(theta1, X_bias.T)
    a2 = sigmoid(g2)
    a2_bias = np.concatenate((np.ones((1, 5000)), a2), axis=0)
    # 10x5000
    g3 = np.matmul(theta2, a2_bias)
    a3 = sigmoid(g3)
    return a3

In [129]:
def costFunction(theta, X, y, reg):    
    y_pred = forwardPropagation(theta, X)
    batch_size = y.shape[0]
    unreg_cost = 0
    for i in range(0, labels):
        idx = (i+9)%10
        unreg_cost = unreg_cost - (1/batch_size)*(np.matmul(np.log(y_pred[idx:idx+1,]), (y==i)*1) + np.matmul(np.log(1-y_pred[idx:idx+1,]), (1-(y==i)*1)))
    reg_cost = (reg*(np.sum(theta1[:,1:]**2) + np.sum(theta2[:,1:]**2)))/(2*batch_size)
    return np.array([unreg_cost[0,0] + reg_cost])

In [130]:
theta = np.concatenate((theta1.flatten(), theta2.flatten()), axis=0)
print("Cost : %f" %costFunction(theta, X, y, reg))

Cost : 0.383770


In [131]:
def giveGradient(theta, X, y, reg):
    theta1 = np.reshape(theta[0:10025], (25, 401))
    theta2 = np.reshape(theta[10025:10285], (10, 26))
    
    batch_size = X.shape[0]
    X_bias = np.concatenate((np.ones((5000,1)), X), axis=1)
    delMat1 = np.zeros((25, 401))
    delMat2 = np.zeros((10, 26))
    for i in range(0, batch_size):
        a1_bias = X_bias[i:i+1,:].T
        z2 = np.matmul(theta1, a1_bias)
        a2 = sigmoid(z2)
        a2_bias = np.concatenate((np.ones((1,1)), a2), axis=0)
        z3 = np.matmul(theta2, a2_bias)
        a3 = sigmoid(z3)
        y_out = np.zeros((10, 1))
        y_out[(y[i,0]+9)%10] = 1
        del3 = a3 - y_out
        del2 = np.matmul(theta2.T, del3)[1:,] * sigmoidGrad(z2)
        delMat1 = delMat1 + np.matmul(del2, a1_bias.T)
        delMat2 = delMat2 + np.matmul(del3, a2_bias.T)
        
    grad1 = (delMat1 + reg*theta1)/batch_size
    grad1[:,0] = grad1[:,0] - (reg*theta1[:,0])/batch_size
    grad2 = (delMat2 + reg*theta2)/batch_size
    grad2[:,0] = grad2[:,0] - (reg*theta2[:,0])/batch_size
    return np.concatenate((grad1.flatten(), grad2.flatten()), axis=0)

In [132]:
def numGradient(theta, X, y, reg):
    epsilon = 0.001
    num_grad = np.zeros(theta.size)
    GRAD = giveGradient(theta, X, y, reg)
    for i in range(0, theta.size):
        theta[i] = theta[i] + epsilon
        J_plus = costFunction(theta, X, y, reg)
        theta[i] = theta[i] - 2*epsilon
        J_minus = costFunction(theta, X, y, reg)
        theta[i] = theta[i] + epsilon
        num_grad[i] = (J_plus - J_minus)/(2*epsilon)
        print(num_grad[i] - GRAD[i])
    
    return num_grad

In [133]:
# print(giveGradient(theta, X, y, 0)[0:30])
# print(numGradient(theta, X, y, 0)[0:30])

In [134]:
epsilon = 0.12
flat_params = np.random.uniform(-epsilon, epsilon, (1, 10285)).flatten()
thetaArr = optimize.fmin_cg(costFunction, flat_params, fprime=giveGradient, args=(X, y, reg), maxiter=50)

         Current function value: 0.430718
         Iterations: 50
         Function evaluations: 108
         Gradient evaluations: 108


In [135]:
y_pred = forwardPropagation(thetaArr, X)
y_pred = np.argmax(y_pred, axis=0)
y_pred = (y_pred + 1)%10
check = (y.flatten() == y_pred)*1
print("Accuracy : %f" %(100*np.sum(check)/(X.shape[0])))

Accuracy : 96.000000


In [ ]:
np.savetxt('text.txt',mat,fmt='%.2f')